In [1]:
import numpy as np
import pandas as pd

In [2]:
# Contains info about each business
businessPd = pd.read_csv("business.csv")
# Contains two columns, one with the test id and the other with the predictions
sampleSub = pd.read_csv('sample_submission.csv')
# Contains two columns, one with the business id and the other with the user id
testPairs = pd.read_csv('test_queries.csv')
# Contains info about past reviews
trainReviews = pd.read_csv('train_reviews.csv')
# Contains info about the ratings that users have given in the past
users = pd.read_csv('users.csv')
# IDK tbh
validate = pd.read_csv('validate_queries.csv')

In [7]:
len(trainReviews)

150232

In [10]:
for index, row in trainReviews.iterrows():
    print index
    print row
    break

0
business_id                               WPCgtEG-bJt0cZtnM-x7yw
cool                                                           0
date                                                  2005-03-04
funny                                                          0
review_id                                 6sferX8QOJq9g5PONKZH4w
stars                                                          4
text           There is nothing better than happy hour on the...
useful                                                         0
user_id                                   VDh1vjzpNUJH6HfcjH8g7Q
Name: 0, dtype: object


In [34]:
pd.set_option('display.max_columns', None)
curID = "WPCgtEG-bJt0cZtnM-x7yw"
t = businessPd[businessPd['business_id'] == curID]

curID = "VDh1vjzpNUJH6HfcjH8g7Q"
p = users[users['user_id'] == curID]
p

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
29271,4.5,0,1,0,0,0,0,0,0,0,0,0,0,None,0,"cT0xyr130ZUpEcj8L7Hjxw, HLaSqQMDVvlcFPGJL_kGCA...",0,Jenn,4,0,VDh1vjzpNUJH6HfcjH8g7Q,2005-03-04


In [51]:
# Create training set 
# Each (x,y) pair
# x = userVector + businessVector
# y = rating

# Creates a vector for each user with the following features
# [avgStars, numReviews]

def createUserDict():
    userDict = {}
    for index, row in users.iterrows():
        avgStars = row['average_stars']
        numReviews = row['review_count']
        userDict[row['user_id']] = [avgStars, numReviews]
    return userDict

# Creates a vector for each business with the following features
# [avgStars, numReviews]

def createBusinessDict():
    businessDict = {}
    for index, row in businessPd.iterrows():
        avgStars = row['stars']
        numReviews = row['review_count']
        businessDict[row['business_id']] = [avgStars, numReviews]
    return businessDict

xTrain = []
yTrain = []
userDict = createUserDict()
businessDict = createBusinessDict()
for index, row in trainReviews.iterrows():
    uv = userDict[row['user_id']]
    bv = businessDict[row['business_id']]
    # Concatenate uv and bv
    newVector = uv + bv
    yTrain.append(row['stars'])
    xTrain.append(newVector)

In [60]:
xTest = []
for index, row in testPairs.iterrows():
    uv = userDict[row['user_id']]
    bv = businessDict[row['business_id']]
    # Concatenate uv and bv
    newVector = uv + bv
    xTest.append(newVector)

In [78]:
from sklearn import linear_model
import sys
import csv

logistic = linear_model.LogisticRegression()
logistic.fit(xTrain, yTrain)
preds = logistic.predict(xTest)

df1 = pd.DataFrame({'labels': preds})
preds = df1['labels'].tolist()
results = [[0 for x in range(2)] for x in range(len(preds))]
for index in range(0,len(preds)):
    results[index][0] = int(index)
    results[index][1] = preds[index]

firstRow = [[0 for x in range(2)] for x in range(1)]
firstRow[0][0] = 'index'
firstRow[0][1] = 'stars'
with open("result.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(firstRow)
    writer.writerows(results)

In [54]:
print np.asarray(xTrain).shape
print np.asarray(yTrain).shape

(150232, 4)
(150232,)


In [3]:
users.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,2.83,0,0,0,0,0,0,0,0,1,...,0,None,0,None,2,James,6,7,UxfpKHGO2dfQCdS9xLLJow,2012-04-25
1,3.00,0,0,0,0,0,0,0,0,0,...,0,None,0,None,0,Yvonne,4,0,Kr5NDQFPPB_01-5CDmSqVg,2015-11-30
2,3.09,0,0,0,0,0,0,0,0,0,...,0,None,1,None,0,Etobicoke,10,2,wfoeMtriLwZsdRzcxNTaFA,2012-11-13
3,4.00,0,0,0,0,0,0,0,0,0,...,0,None,0,None,0,Shirley,4,0,aXb0kCIsIbPEEUSGomrrmA,2014-09-10
4,4.00,0,0,0,0,0,0,0,0,0,...,0,None,0,None,0,Brian,1,0,sLrX2KGu3lc_JczAnsg0_Q,2015-03-22


In [39]:
businessPd.head()

,address,attributes,attributes_AcceptsInsurance,attributes_AgesAllowed,attributes_Alcohol,attributes_Ambience,attributes_BYOB,attributes_BYOBCorkage,attributes_BestNights,attributes_BikeParking,attributes_BusinessAcceptsBitcoin,attributes_BusinessAcceptsCreditCards,attributes_BusinessParking,attributes_ByAppointmentOnly,attributes_Caters,attributes_CoatCheck,attributes_Corkage,attributes_DietaryRestrictions,attributes_DogsAllowed,attributes_DriveThru,attributes_GoodForDancing,attributes_GoodForKids,attributes_GoodForMeal,attributes_HairSpecializesIn,attributes_HappyHour,attributes_HasTV,attributes_Music,attributes_NoiseLevel,attributes_Open24Hours,attributes_OutdoorSeating,attributes_RestaurantsAttire,attributes_RestaurantsCounterService,attributes_RestaurantsDelivery,attributes_RestaurantsGoodForGroups,attributes_RestaurantsPriceRange2,attributes_RestaurantsReservations,attributes_RestaurantsTableService,attributes_RestaurantsTakeOut,attributes_Smoking,attributes_WheelchairAccessible,attributes_WiFi,business_id,categories,city,hours,hours_Friday,hours_Monday,hours_Saturday,hours_Sunday,hours_Thursday,hours_Tuesday,hours_Wednesday,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,595 Markham Street,NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,NaN,True,"{'garage': False, 'street': True, 'validated':...",NaN,True,NaN,NaN,NaN,False,NaN,NaN,False,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,False,NaN,average,NaN,True,casual,NaN,False,True,2.0,True,True,True,NaN,False,no,KuxDPl6UYNLxFChPm0_MNw,"Cajun/Creole, Southern, Restaurants",Toronto,NaN,17:0-1:0,NaN,17:0-1:0,17:0-1:0,17:0-1:0,17:0-1:0,17:0-1:0,0,43.664125,-79.411886,Southern Accent Restaurant,Palmerston,M6G 2L7,146,4.0,ON
1,2801 N 15th Ave,NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",False,yes_free,"{'monday': False, 'tuesday': True, 'friday': T...",True,NaN,True,"{'garage': False, 'street': False, 'validated'...",NaN,False,False,False,NaN,NaN,False,False,True,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,True,"{'dj': False, 'background_music': False, 'no_m...",average,NaN,True,casual,NaN,False,True,1.0,False,False,True,outdoor,True,no,6SAfQKe2oM5g_EtcYXyAMg,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,NaN,11:0-23:0,11:0-22:0,11:0-22:0,11:0-21:0,11:0-22:0,11:0-22:0,11:0-22:0,1,33.479807,-112.091188,Original Hamburger Works,NaN,85007,277,4.0,AZ
2,"5508 County Rd N, Ste 3",NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,NaN,False,"{'garage': False, 'street': False, 'validated'...",NaN,False,False,NaN,NaN,NaN,NaN,False,False,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,True,"{'dj': False, 'background_music': False, 'no_m...",average,NaN,True,casual,NaN,False,True,1.0,False,False,True,outdoor,True,no,upB0RQl-l529IVwgOpwOQQ,"Nightlife, Restaurants, Bars, Chicken Wings, A...",Sun Prairie,NaN,11:0-0:0,11:0-0:0,11:0-0:0,11:0-0:0,11:0-0:0,11:0-0:0,11:0-0:0,1,43.149488,-89.206641,Chicken Lips,NaN,53590,102,4.5,WI
3,2227 N Rampart Blvd,NaN,NaN,NaN,beer_and_wine,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,NaN,True,"{'garage': False, 'street': False, 'validated'...",NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,True,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,True,NaN,average,NaN,False,casual,NaN,False,True,2.0,False,True,True,NaN,True,no,TulmRC5V0--dnXYd_GOSvA,"Beer, Wine & Spirits, Italian, Food, American ...",Las Vegas,NaN,7:0-19:0,7:0-14:30,7:0-14:30,7:0-14:30,7:0-19:0,7:0-19:0,7:0-19:0,1,36.201990,-115.283122,Omelet House Summerlin,Summerlin,89128,242,4.0,NV
4,1111 W Bell Rd,NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,NaN,True,"{'garage': False, 'street': False, 'validated'...",NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,True,NaN,average,NaN,False,ca

In [40]:
businessPd.columns

Index([u'address', u'attributes', u'attributes_AcceptsInsurance',
       u'attributes_AgesAllowed', u'attributes_Alcohol',
       u'attributes_Ambience', u'attributes_BYOB', u'attributes_BYOBCorkage',
       u'attributes_BestNights', u'attributes_BikeParking',
       u'attributes_BusinessAcceptsBitcoin',
       u'attributes_BusinessAcceptsCreditCards', u'attributes_BusinessParking',
       u'attributes_ByAppointmentOnly', u'attributes_Caters',
       u'attributes_CoatCheck', u'attributes_Corkage',
       u'attributes_DietaryRestrictions', u'attributes_DogsAllowed',
       u'attributes_DriveThru', u'attributes_GoodForDancing',
       u'attributes_GoodForKids', u'attributes_GoodForMeal',
       u'attributes_HairSpecializesIn', u'attributes_HappyHour',
       u'attributes_HasTV', u'attributes_Music', u'attributes_NoiseLevel',
       u'attributes_Open24Hours', u'attributes_OutdoorSeating',
       u'attributes_RestaurantsAttire',
       u'attributes_RestaurantsCounterService',
       u'attr

In [5]:
sampleSub.head()

,index,stars
0,0,5.0
1,1,5.0
2,2,5.0
3,3,5.0
4,4,5.0


In [15]:
testPairs['user_id'].value_counts()

UHAgUQn4FNO56RVHFdxEeQ    249
VbHUkjEWOTQ_ZDHZz33lQQ    140
wGI-TPM4P_QRCF2BHdJCkg    137
L4qJbGNBCo2567HHLcsUXA    135
awAYFR8liRE8HimuzhpE2w    124
rLxuEIbYiEqcT8zHkruiTA    108
7UOGJUyi33IgURfRawe1Yw    101
hVp7Ls-IW2zpbfhb5R_rSg    100
0IGpZVUasT3fubxFSWPGcA     99
rzG4fvakx5EmYFGEH6C4RQ     97
y8jJV4AeH6ZEBagQ_ys8KQ     95
6vK-tl-9Y-8fRSjIU2tSMQ     92
8erAzkOo9KHs1Njo5bMm_Q     90
BC0kO_UYy7iSV6aIsMa6Sw     89
sjgCe1gytIKQHHIISfRH6w     84
ACUVZ4SiN0gni7dzVDm9EQ     83
WdZXKWoC9OC72kkvHZhUgA     79
Onr-WMCwREXOp5-yoMk-iQ     74
nYRGkhifp4bPm0r2uZ2fSg     70
DCj47Eu0IDNowUyWlm3Fog     68
Kb-kFHW1NBwsN1p8_ppwkg     66
Vc-w8-7QIrrN07oveskT3g     66
Y6aJzFjeQTkydnazXsPwiQ     65
5Jni-Yc-KP71yN1jevskvQ     65
lJ0iMJ9VsYEtXd7qbqShCg     65
GL0o95jlTawH53za2DRKNQ     64
eRvPOtsjkzTjn0f__HaXKQ     64
668vgzWmebSePL87B3xJ4A     64
MPGqUhAsmVEHq9k9X6VZKw     63
oWuINpXVECuBfrZ3nrPtgw     63
                         ... 
ccW-djwIV382raCE6P_eOw      1
mtGwkN52EPd6vGwq9JBBCg      1
c5uSmULn-X

In [10]:
businessPd.head()

,address,attributes,attributes_AcceptsInsurance,attributes_AgesAllowed,attributes_Alcohol,attributes_Ambience,attributes_BYOB,attributes_BYOBCorkage,attributes_BestNights,attributes_BikeParking,...,hours_Wednesday,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,595 Markham Street,NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,17:0-1:0,0,43.664125,-79.411886,Southern Accent Restaurant,Palmerston,M6G 2L7,146,4.0,ON
1,2801 N 15th Ave,NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",False,yes_free,"{'monday': False, 'tuesday': True, 'friday': T...",True,...,11:0-22:0,1,33.479807,-112.091188,Original Hamburger Works,NaN,85007,277,4.0,AZ
2,"5508 County Rd N, Ste 3",NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,...,11:0-0:0,1,43.149488,-89.206641,Chicken Lips,NaN,53590,102,4.5,WI
3,2227 N Rampart Blvd,NaN,NaN,NaN,beer_and_wine,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,7:0-19:0,1,36.201990,-115.283122,Omelet House Summerlin,Summerlin,89128,242,4.0,NV
4,1111 W Bell Rd,NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,11:0-22:0,1,33.639774,-112.087738,Manuel's Mexican Restaurant & Cantina - Bell Rd,NaN,85023,230,3.5,AZ


In [13]:
trainReviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,WPCgtEG-bJt0cZtnM-x7yw,0,2005-03-04,0.0,6sferX8QOJq9g5PONKZH4w,4.0,There is nothing better than happy hour on the...,0.0,VDh1vjzpNUJH6HfcjH8g7Q
1,LnnO7quTjjdTUkCshSJnkA,2,2005-07-12,0.0,OybsTOFlklnoFl8ZwRa_AQ,5.0,You know you're in for a fun experience as soo...,3.0,HnnjIuLrdhLTsRRVrrFIjA
2,sKrlmbrZWCyLIgiMihCPqw,0,2005-07-12,0.0,fE-Z39pw4Mb5yjr9GihtHQ,5.0,"Vegas has become a massive theme park, where e...",3.0,HnnjIuLrdhLTsRRVrrFIjA
3,Lh5qnT2m2b4lvyYiMGMDkg,1,2005-07-12,0.0,DuC9JQLQ3yY0kPsonFvk-Q,4.0,Having stayed at most of the theme park-like V...,3.0,HnnjIuLrdhLTsRRVrrFIjA
4,54LYVM1gCGQ2UVFK9QhgTw,3,2005-07-12,0.0,cUFJE_U6s6q9_tjDw3nTmA,5.0,"Andre's Monte Carlo, voted the #1 restaurant i...",6.0,HnnjIuLrdhLTsRRVrrFIjA


In [11]:
users.columns

Index([u'average_stars', u'compliment_cool', u'compliment_cute',
       u'compliment_funny', u'compliment_hot', u'compliment_list',
       u'compliment_more', u'compliment_note', u'compliment_photos',
       u'compliment_plain', u'compliment_profile', u'compliment_writer',
       u'cool', u'elite', u'fans', u'friends', u'funny', u'name',
       u'review_count', u'useful', u'user_id', u'yelping_since'],
      dtype='object')

In [12]:
businessPd.columns

Index([u'address', u'attributes', u'attributes_AcceptsInsurance',
       u'attributes_AgesAllowed', u'attributes_Alcohol',
       u'attributes_Ambience', u'attributes_BYOB', u'attributes_BYOBCorkage',
       u'attributes_BestNights', u'attributes_BikeParking',
       u'attributes_BusinessAcceptsBitcoin',
       u'attributes_BusinessAcceptsCreditCards', u'attributes_BusinessParking',
       u'attributes_ByAppointmentOnly', u'attributes_Caters',
       u'attributes_CoatCheck', u'attributes_Corkage',
       u'attributes_DietaryRestrictions', u'attributes_DogsAllowed',
       u'attributes_DriveThru', u'attributes_GoodForDancing',
       u'attributes_GoodForKids', u'attributes_GoodForMeal',
       u'attributes_HairSpecializesIn', u'attributes_HappyHour',
       u'attributes_HasTV', u'attributes_Music', u'attributes_NoiseLevel',
       u'attributes_Open24Hours', u'attributes_OutdoorSeating',
       u'attributes_RestaurantsAttire',
       u'attributes_RestaurantsCounterService',
       u'attr

In [18]:
businessPd['stars']

0        4.0
1        4.0
2        4.5
3        4.0
4        3.5
5        4.0
6        3.0
7        4.0
8        3.5
9        4.0
10       1.5
11       3.5
12       4.0
13       4.0
14       3.5
15       3.0
16       4.0
17       4.5
18       4.0
19       3.5
20       4.0
21       3.5
22       3.5
23       4.5
24       3.0
25       3.5
26       3.5
27       3.5
28       3.0
29       3.5
        ... 
12028    4.0
12029    4.0
12030    3.5
12031    3.5
12032    4.0
12033    4.5
12034    4.0
12035    4.5
12036    4.0
12037    3.5
12038    2.5
12039    5.0
12040    4.0
12041    4.5
12042    4.0
12043    4.5
12044    3.5
12045    3.5
12046    3.5
12047    5.0
12048    2.5
12049    4.5
12050    4.0
12051    4.5
12052    4.5
12053    4.0
12054    4.5
12055    4.5
12056    4.0
12057    4.0
Name: stars, Length: 12058, dtype: float64